# What can pyhasse.acm be used for?

### Objectives

Analysis of conflicts (Antichain: a subset of the set of objects, mutually incomparable)

Note: only the representative elements are considered. 

For example: In the data file csvdata/a-equiv.txt are 8 objects but only 5 representative elements

### Let's calculate the colsum

Number of incomparable object pairs, concerning a given indicator pair (col sum).


### Import all necessary libraries

In [1]:
import pathlib
from pyhasse.core.csv_io import CSVReader
from pyhasse.core.matrix import Matrix
import json
import random
from IPython.core.display import display, HTML
from string import Template
import hd3d_lib
from pyhasse.acm.calc import ACM

### load library for visualisation

In [2]:
HTML('<script src="lib/d3/d3.min.js"></script>')

### Note:

start: all bars whose length >= 'start' will be visible

stop:  all bars whose length <= 'stop' will be shown

In [3]:
TESTFILENAME = '/csvdata/a-equiv.txt'
#TESTFILENAME = '/csvdata/check-K.csv'
prefs={
    'start': 0,
    'stop': 20,
    'user_list' : [],
    #'user_list' : ['a', 'b', 'g'],
    'labelLen': 3,
    'actwidth': 550,
    'actheight': 200
}

HERE = pathlib.Path('__file__').parent
csv = CSVReader(fn=str(HERE) + TESTFILENAME, ndec=3)
# prepare dataset (reduced system)
matrix = Matrix(csv.data, csv.obj, csv.prop, reduced=True)
acm = ACM(matrix, csv)

#acm = ACM(csv.data, csv.rows, csv.cols, reduced=True)
print(csv.rows)
print(csv.cols)
print(csv.obj)
prefs['user_list'] = csv.obj

8
3
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']


#### Equivalenceclasses

In [4]:
eqm, rred =  matrix.generate_equivalenceclasses(csv.data, csv.rows, csv.cols, prec=None)
print(f'{"Pointer":>8} {"object label represents":^30} {"equivalence class":^20}')
print(f'{59 * "-"}')
for idx, value in enumerate(matrix.obj): 
      print(f'{csv.obj.index(value):>8} {value:^30} {str(eqm[idx]):^20}')   

 Pointer    object label represents      equivalence class  
-----------------------------------------------------------
       0               a                     [0, 1, 5]      
       2               c                        [2]         
       3               d                        [3]         
       4               e                       [4, 6]       
       7               h                        [7]         


 ### Basic calculations


In [5]:
px, pq = acm.generate_setofpairs(prefs['user_list'])
ac = acm.calc_acm(px, pq)
print (csv.data)
print ("px ",px)
print ("pq ",pq)
#print ("qp ",qp)
print ("ac ",ac)
rowsumacm, colsumacm = acm.calc_obj_attprofile()
maxrowsum, maxcolsum = acm.calc_optimum()
mobjobj, mattatt = acm.find_optimalpairs(px, pq)

print (csv.obj)

for el in px:
    print (el)

[[1.0, 1.0, 2.0], [1.0, 1.0, 2.0], [3.0, 2.0, 1.0], [1.0, 2.0, 3.0], [4.0, 4.0, 4.0], [1.0, 1.0, 2.0], [4.0, 4.0, 4.0], [9.0, 9.0, 9.0]]
px  [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]
pq  [(0, 1), (0, 2), (1, 2)]
ac  [[0, 0, 0], [0, 1, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0], [0, 1, 1], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(1, 2)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 7)
(2, 3)
(2, 4)
(2, 5)
(2, 6)
(2, 7)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(4, 5)
(4, 6)
(4, 7)
(5, 6)
(5, 7)
(6, 7)


In [6]:
name_ordinate1, \
name_ordinate2, \
labels_obj, \
labels_att, \
labelmaxobj, \
labelmaxatt = acm.acm_graphics(prefs['user_list'])

#labels_obj
#labels_att

### Prepare the data for visualisation

In [7]:
rowsum, colsum = acm.calc_obj_attprofile()
print(colsum)
print(rowsum)

[0, 4, 3]
[0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Convert data into javscript syntax for D3

In [8]:
# min and max from colsum
#print(csv.obj)
results_list = sorted(colsum)
print(results_list)
# prepare javascript variables
data = ''
for k,v in prefs.items():
    data += "var {} = {};\n".format(k,v)

tmpl = '{{"legendLabel": "{0}", "magnitude": {1}}},'
ll = prefs['labelLen']


data += 'var dataSet1  = ['
for x in range(0, len(colsum)):
    if colsum[x] in range(prefs['start'], prefs['stop']+1):
        l1 = labels_att[x][0][:ll]
        l2 = labels_att[x][1][:ll]
        value = str(colsum[x])
        data += tmpl.format("{} - {}".format(l1, l2), value)
data += "];"
print(data)

[0, 3, 4]
var start = 0;
var stop = 20;
var user_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h'];
var labelLen = 3;
var actwidth = 550;
var actheight = 200;
var dataSet1  = [{"legendLabel": "q1 - q2", "magnitude": 0},{"legendLabel": "q1 - q3", "magnitude": 4},{"legendLabel": "q2 - q3", "magnitude": 3},];


### Lets call a D3-Graph with our data

In [9]:
HTML(hd3d_lib.draw_graph('acm_colsum',{'data': data}))